# Base parameters and combinations with QR

In the following, the QR decomposition is used to find columns in a matrix (e.g. dynamics regressor) that are independent and therefore are a choice of a basis for the identifiable subspace. It is then determined which linear combinations of the other dependent columns are forming the base columns/parameters in each of those. This is also necessary for the std essential parameters.

In [1]:
import numpy as np; np.set_printoptions(formatter={'float': '{: 0.3f}'.format})

In [2]:
a = np.array([[1.0, 2.0,  .0,  5.0, 2.0, 3.0],
              [1.0, 2.0,  .0,  4.0, 1.5, 2.5],
              [ .0,  .0, 1.0,  .0,   .0, .0],
              [ .0,  .0, 1.0,  .0,   .0, .0],
              [2.0, 4.0,  .0,  6.0,  .0, 6.0],
              [ .0,  .0, 0.5,  0.5, 0.5, .0]])
print(a.shape)
r = np.linalg.matrix_rank(a)
r

(6, 6)


4

Test Matrix $a$ is rank deficient (rank $r<m$). The rank 4 means there are 4 sets of dependent columns, each with 1 or more columns in it. The columns $0,1$ and $3,4,5$ are linearly dependent. Column 2 is linearly independent from all other columns. Finding independent columns will give a single representant of each set, while the choice is non-unique if any set has more than one element. If a column is independent, there can be other columns that are dependent on it but it can also be a single element in its set.

Assuming the set of linear equations, x being known parameters.
$$A\cdot x = t$$

In [32]:
x = np.array([1.0,2.0,3.0,4.0,4.5,2.0])  #param
t = a.dot(x)                         #result
t

array([ 40.000,  32.750,  3.000,  3.000,  46.000,  5.750])

## Sets of dependent columns


Find sets of columns that are linearly independent to each other. For non-pivoting QR, the columns in R with diagonal elements that are not zero are "independent", i.e. these columns are selected as representative for this set with factor 1. For pivoting QR it does not work the same way. As there are as many "independent" sets as the rank of the matrix, taking the first rank columns of the permutation also gives the independent column indices but in different order. The choices for the representatives are possibly not the same but from the same independent sets.

Assuming $m \times n$ Matrix $a$ with $m>=n$.

In [33]:
import scipy.linalg as sla
Qp,Rp,P = sla.qr(a, pivoting=True, check_finite=True)
print(Rp.diagonal())
print(P)
#print()
#Q,R = np.linalg.qr(a, mode='complete')
#print(R.diagonal())

[-8.789 -1.755 -1.496 -0.269 -0.000 -0.000]
[3 4 2 1 5 0]


In [54]:
tol = 1e-6
#print(R.diagonal())
#indepsQR = np.where(np.abs(R.diagonal()) > tol)[0]
#print(indepsQR)

print("rank: {}".format(np.where(np.abs(Rp.diagonal()) > tol)[0].size ))   #rank with pivoting QR
indepsQRP = P[0:r]
print(indepsQRP)

rank: 4
[3 4 2 1]


In [35]:
#create proper permutation matrix
Pp = np.zeros((P.size, P.size))
for i in P:
    Pp[i,P[i]]=1

In [76]:
#Gautier 1990, 3-4: linear relationships
#W1W2 = a.dot(Pp)
#W1 = W1W2[:,0:indepsQRP.size]
#W2 = W1W2[:,indepsQRP.size:a.shape[1]]
#Q,R = np.linalg.qr(W1W2)
#print(Q)
#print(R.diagonal())
#corresponds to QR with pivoting
#print(Qp)
#print(Rp.diagonal())

#dependent columns W1 as linear combinations of independent columns W1
R1 = Rp[0:indepsQRP.size,0:indepsQRP.size]
R2 = Rp[0:indepsQRP.size,indepsQRP.size:]
linDeps = np.linalg.inv(R1).dot(R2)
print(linDeps)
print(indepsQRP)
#should be equal
#print(W2)
#print(W1.dot(linDeps))

[[ 1.000 -0.000]
 [-1.000  0.000]
 [ 0.000  0.000]
 [ 0.000  0.500]]
[3 4 2 1]


###  Getting the dependencies expressed for each of the independent columns

In [155]:
print("P:", P)
print("a:\n", a)

from sympy import symbols, solve
syms = symbols('c0:%d' % a.shape[1])

# collect dependencies expressed from dependent columns (coming like that from QR)
indep_eqs = list()
for j in range(0,linDeps.shape[1]):
    dep_org_col = P[indepsQRP.size+j]
    # build linear equation for this dependent column
    eq = 0
    #go through all dependencies of this col and collect all grouped params
    for i in range(0,linDeps.shape[0]):   
        if np.abs(linDeps[i,j]) > 0.01:
            indep_org_col = P[i]
            eq += round(linDeps[i,j], 4)*syms[indep_org_col]
            
    eq = Eq(syms[dep_org_col], eq)
    print("{} = {}".format(eq.lhs, eq.rhs))
    indep_eqs.append(eq)

# collect grouped columns for each independent column
indeps_deps = {}
for j in indepsQRP:
    for i in range(0,linDeps.shape[1]):
        #print(syms[j], i)
        if (not j in indeps_deps or indeps_deps[j] == []) and syms[j] in indep_eqs[i].free_symbols:
            indeps_deps[j] = solve(indep_eqs[i], syms[j])
            #print("solving {}".format(indep_eqs[i].free_symbols))
        elif j in indeps_deps and indeps_deps[j] != []:
            break
print(indeps_deps)

P: [3 4 2 1 5 0]
a:
 [[ 1.000  2.000  0.000  5.000  2.000  3.000]
 [ 1.000  2.000  0.000  4.000  1.500  2.500]
 [ 0.000  0.000  1.000  0.000  0.000  0.000]
 [ 0.000  0.000  1.000  0.000  0.000  0.000]
 [ 2.000  4.000  0.000  6.000  0.000  6.000]
 [ 0.000  0.000  0.500  0.500  0.500  0.000]]
c5 = 1.0*c3 - 1.0*c4
c0 = 0.5*c1
{1: [2.0*c0], 3: [c4 + c5], 4: [c3 - c5]}


## Getting the base projection matrix

The first $r$ orthogonal columns of $Q$ form a basis $B$ of the column space of $A$ in case it is a square matrix. In the rectangular case, the QR decomposition of $A^T$ seems to be possible but maybe not always solvable?. 
$B$ can be used to project the matrix $A$ with regard to a vector $v$ to a matrix $A_\text{base}$ with regard to a vector $v_\text{base}$.

In [45]:
Qpt,Rpt,Pt = sla.qr(a.T, pivoting=True, mode='economic')
print(Qpt)
B = -Qpt[:, 0:r]
print(B)
#B = sla.orth(a)[0:a.shape[1], :]
#print(B)

[[-0.209 -0.103  0.000  0.382  0.852 -0.271]
 [-0.417 -0.206  0.000  0.764 -0.426  0.136]
 [-0.000  0.000  1.000  0.000  0.000  0.000]
 [-0.626  0.480  0.000 -0.212 -0.175 -0.550]
 [-0.000  0.788  0.000  0.212  0.175  0.550]
 [-0.626 -0.309 -0.000 -0.424  0.175  0.550]]
[[ 0.209  0.103 -0.000 -0.382]
 [ 0.417  0.206 -0.000 -0.764]
 [ 0.000 -0.000 -1.000 -0.000]
 [ 0.626 -0.480 -0.000  0.212]
 [ 0.000 -0.788 -0.000 -0.212]
 [ 0.626  0.309  0.000  0.424]]


$Pp^T XB = [XB1 ; XB2] = [X1 + X1^{-1}R2 X2;0]$

In [46]:
print(P)
Px = Pp.dot(x)
print('x:\t', x)
x1 = Px[0:r]
x2 = Px[r:]
print ('x1, x2:\t', x1, x2)
xb1 = x1 + np.linalg.inv(R1).dot(R2.dot(x2))
print (xb1)

[3 4 2 1 5 0]
x:	 [ 1.000  2.000  3.000  4.000  4.500  2.000]
x1, x2:	 [ 4.000  4.500  3.000  2.000] [ 2.000  1.000]
[ 6.000  2.500  3.000  2.500]


In [47]:
a_base = a.dot(B)
x_base = B.T.dot(x)
print('x_base:\t', x_base)
print("deps:")
for i in range(0, len(x_base)):
    print("{}, ".format(indepsQRP[i], deps[i]))
t_base = a_base.dot(x_base)
print('t:\t', t)
print('t_base:\t', t_base)
print('x:\t', x)
print ('x*:\t', B.dot(x_base))    #should give same x as in standard space if x was fully in base space already

# test nullspace
print(a.dot(np.eye(6) - B.dot(B.T)))

x_base:	 [ 4.796 -4.336 -3.000 -1.167]
deps:
3, 
4, 
2, 
1, 
t:	 [ 40.000  32.750  3.000  3.000  46.000  5.750]
t_base:	 [ 40.000  32.750  3.000  3.000  46.000  5.750]
x:	 [ 1.000  2.000  3.000  4.000  4.500  2.000]
x*:	 [ 1.000  2.000  3.000  4.833  3.667  1.167]
[[ 0.000  0.000  0.000  0.000  0.000  0.000]
 [ 0.000  0.000  0.000  0.000 -0.000  0.000]
 [ 0.000  0.000  0.000  0.000  0.000  0.000]
 [ 0.000  0.000  0.000  0.000  0.000  0.000]
 [ 0.000  0.000  0.000  0.000  0.000  0.000]
 [ 0.000  0.000  0.000 -0.000  0.000  0.000]]


In [48]:
deps = np.linalg.inv(Rpt[0:r, 0:r]).dot(Rpt[0:r, r:])
print(deps)

[[-0.125  0.000]
 [ 0.250  0.000]
 [ 0.500  1.000]
 [ 0.000  0.000]]


### Base Projection with SVD

In [44]:
U, s, Vh = sla.svd(a, full_matrices=False)
print(U)
print(Vh)
np.sum(s>0.001)

[[-0.506  0.609  0.081 -0.560  0.223  0.049]
 [-0.424  0.391  0.055  0.815 -0.000  0.000]
 [-0.000  0.031 -0.668  0.030  0.373 -0.642]
 [-0.000  0.031 -0.668  0.030  0.072  0.739]
 [-0.750 -0.642 -0.072 -0.077 -0.111 -0.024]
 [-0.033  0.248 -0.305 -0.116 -0.891 -0.194]]
[[-0.193 -0.386 -0.001 -0.694 -0.132 -0.562]
 [-0.116 -0.232  0.075  0.358  0.785 -0.427]
 [-0.006 -0.011 -0.997  0.026  0.061 -0.035]
 [ 0.386  0.773  0.008 -0.238  0.171 -0.409]
 [-0.592  0.296  0.000  0.433 -0.433 -0.433]
 [-0.670  0.335 -0.000 -0.382  0.382  0.382]]


4

In [49]:
# QR base for comparison
B

array([[ 0.209,  0.103, -0.000, -0.382],
       [ 0.417,  0.206, -0.000, -0.764],
       [ 0.000, -0.000, -1.000, -0.000],
       [ 0.626, -0.480, -0.000,  0.212],
       [ 0.000, -0.788, -0.000, -0.212],
       [ 0.626,  0.309,  0.000,  0.424]])

In [43]:
B_svd = -Vh.T[:, 0:r]
B = B_svd
B

array([[ 0.193,  0.116,  0.006, -0.386],
       [ 0.386,  0.232,  0.011, -0.773],
       [ 0.001, -0.075,  0.997, -0.008],
       [ 0.694, -0.358, -0.026,  0.238],
       [ 0.132, -0.785, -0.061, -0.171],
       [ 0.562,  0.427,  0.035,  0.409]])